MODULOS

In [1]:
import pyodbc
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

Conexion 


In [2]:
#Creo la conexion
server = 'GINO-PC-FIJA'
base = 'Ventas2'
Cadena = "DRIVER=ODBC Driver 17 for SQL Server;SERVER= {};DATABASE={};Trusted_Connection=yes".format(server,base)
#Compruebo que funciona
try:
    cnxn = pyodbc.connect(Cadena)
    cursor = cnxn.cursor()
    cursor = cnxn.execute('SELECT 1+1')
except Exception as e:
        print(e)
finally:
    print('Conexion establecida') 

Conexion establecida


Select

In [4]:
#FUNCIONES
def CreadorDF_vent_vendedor(vendedor):
	# Creo el select un una variable 
	query = """SELECT 
				YEAR(vc.fecha) as Año,
				COUNT(vc.factura) as Total
			FROM
				vencab vc
				inner join vendedores v on (v.vendedor = vc.vendedor)
				inner join vendet vt on (vc.letra = vt.letra and vc.factura = vt.factura)
			WHERE
				vc.anulada = 0
				and v.vendedor = {}
			GROUP BY
				YEAR(vc.fecha)
			Order by
				1""".format(vendedor)
	#Global es una funcion que crea una variable que no muere cuando se acaba la funcion
	global df
	#Meto la variable y la conexion al servidor en esta funcion de Pandas
	#PD: podrias poner directamente el texto sql adentro donde dice query por si queres probar algo
	df = pd.read_sql(query, cnxn)
def CreadorDF_vent_sucursal(sucursal):
	# Creo el select un una variable 
	query = """SELECT
				YEAR(vc.fecha) as Año,
				COUNT(vc.factura) as 'Total Ventas'
			FROM vendet vt
				inner join vencab vc on (vt.letra = vc.letra and vt.factura = vc.factura)
				inner join sucursales s on (vc.sucursal = s.sucursal )
			WHERE
				vc.anulada = 0
				and s.sucursal = {}
			GROUP BY
				YEAR(vc.fecha)
			ORDER BY
				1,2""".format(sucursal)
	#Global es una funcion que crea una variable que no muere cuando se acaba la funcion
	global df
	#Meto la variable y la conexion al servidor en esta funcion de Pandas
	#PD: podrias poner directamente el texto sql adentro donde dice query por si queres probar algo
	df = pd.read_sql(query, cnxn)
def CreadorDF_vent_marcas(marca):
	# Creo el select un una variable 
	query = """SELECT
				YEAR(vc.fecha) as Año,
				COUNT(vc.factura) as 'Cantidad Venta'
			FROM articulos a
				inner join marcas m on (m.marca = a.marca)
				inner join vendet vt on (vt.articulo = a.articulo)
				inner join vencab vc on (vc.letra = vt.letra and vc.factura = vt.factura)
			WHERE
				vc.anulada = 0
				and m.marca = '{}'
			GROUP BY
				YEAR(vc.fecha)
			Order by
				1,2""".format(marca)
	#Global es una funcion que crea una variable que no muere cuando se acaba la funcion
	global df
	#Meto la variable y la conexion al servidor en esta funcion de Pandas
	#PD: podrias poner directamente el texto sql adentro donde dice query por si queres probar algo
	df = pd.read_sql(query, cnxn)
def CreadorDF_vent_articulo(articulo):
	# Creo el select un una variable 
	query = """select
				year(vc.fecha) as "Año"
				,sum(vd.cantidad) as "Ventas"
			from
				vencab as vc
				inner join vendet as vd
				on vd.factura = vc.factura and vd.letra = vc.letra
			where
				vc.anulada=0
				and vd.articulo = '{}'
			group by
				year(vc.fecha)""".format(articulo)
	#Global es una funcion que crea una variable que no muere cuando se acaba la funcion
	global df
	#Meto la variable y la conexion al servidor en esta funcion de Pandas
	#PD: podrias poner directamente el texto sql adentro donde dice query por si queres probar algo
	df = pd.read_sql_query(query,cnxn)

In [22]:
# Creo el saludo y le doy instrucciones al Usuario
print('-----------------------------------------------------------------------')
print('                            Bienvenido')
print('-----------------------------------------------------------------------')
print('-Este programa Puede Realizar 4 pronosticaciones.')
print('-----------------------------------------------------------------------')
print('-1 Cantidad de ventas por Vendedor.')
print('-----------------------------------------------------------------------')
print('-2 Cantidad de ventas por Sucursal.')
print('-----------------------------------------------------------------------')
print('-3 Cantidad de ventas por Marca.')
print('-----------------------------------------------------------------------')
print('-4 Cantidad de ventas por Articulo.')
print('-----------------------------------------------------------------------')
print('Respectivamente el programa las reconoce a las mismas como 1, 2, 3, 4.')
print('-----------------------------------------------------------------------')
#Creo la variable que toma los datos del Usuario
Elec = int(input('Introduzca la funcion a realizar: '))
if Elec == 1:
    print('Eligío: 1 -Cantidad de ventas por Vendedor')
    print('-----------------------------------------------------------------------')
    #Hago que el Usuario introduzca al Vendedor
    vendedor = int(input('Ahora necesito que elija ¿a cual vendedor desea pronosticar?'))
    #Llamo la funcion que cree antes para crear el DF con la query de SQL
    CreadorDF_vent_vendedor(vendedor)
    #Creo y entreno el Modelo y se lo asigno a la variable LR
    LR = LinearRegression().fit(df['Año'].values.reshape(-1,1),df['Total'])
    #Informo al usuario que ahora debe elegir el año que desea conocer y le doy algunas advertencias
    print('¿Ahora que año desea pronosticar?')
    print('------------------------------------------------------------------------------------------------------------')
    print('Tome en cuenta que mientras mas lejos del ultimo año que se tienen datos, mas especulativos seran los mismos')
    print('Ademas que mientras menos datos se tenga es mas probable que la pronosticacion sea inexacta')
    print('------------------------------------------------------------------------------------------------------------')
    print('Aca te muestro la lista de los años que tenemos datos de este vendedor y sus valores')
    #Muestro el DF que traje para que el usuario conozca con lo que esta trabajando
    print(str(df))
    #Hago que el usuario introduzca el Año que desea predecir
    A = int(input('Introduce el año a predecir: '))
    #Creo la funcion que predice el año con una regresion lineal en base a los datos obtenidos del entrenamineto
    y_pred = LR.intercept_ + LR.coef_* A
    #Muestro el resultado de esa operacion
    print('La estimacion para el Año {} es de: '.format(A)+str(int(y_pred)) + ' Ventas en total para el vendedor {}'.format(vendedor))
elif Elec == 2:
    print('Eligío 2 -Cantidad de ventas por Sucursal')
    print('-----------------------------------------------------------------------')
    #Hago que el Usuario introduzca a la Sucursal
    Sucursal = int(input('Ingrese el numero de la Sucursal a pronosticar: '))
    #Llamo la funcion que cree antes para crear el DF con la query de SQL
    CreadorDF_vent_sucursal(Sucursal)
    #Creo y entreno el Modelo y se lo asigno a la variable LR
    LR = LinearRegression().fit(df['Año'].values.reshape(-1,1),df['Total Ventas'])
    #Informo al usuario que ahora debe elegir el año que desea conocer y le doy algunas advertencias
    print('¿Ahora que año queres pronosticar?')
    print('------------------------------------------------------------------------------------------------------------')
    print('Toma en cuenta que mientras mas lejos del ultimo año que se tienen datos, mas especulativos seran los datos')
    print('Ademas que mientras menos datos se tenga es mas probable que la pronosticacion sea inexacta')
    print('------------------------------------------------------------------------------------------------------------')
    print('Aca te muestro la lista de los años que tenemos datos de esta sucursal y sus valores')
    #Muestro el DF que traje para que el usuario conozca con lo que esta trabajando
    print('\n'+str(df)+'\n')
    #Hago que el usuario introduzca el Año que desea predecir
    A = int(input('Introduce el año a predecir: '))
    #Creo la funcion que predice el año con una regresion lineal en base a los datos obtenidos del entrenamineto
    y_pred = LR.intercept_ + LR.coef_* A
    #Muestro el resultado de esa operacion
    print('La estimacion para el Año {} es de: '.format(A)+str(int(y_pred)) + ' Ventas en total para la Sucursal {}'.format(Sucursal))
elif Elec == 3:
    print('Eligío 3 Cantidad de ventas por Marca')
    print('-----------------------------------------------------------------------')
    #Hago que el Usuario introduzca a la Marca
    Marca = str(input('Ingrese el numero de la Marca a pronosticar: '))
    #Llamo la funcion que cree antes para crear el DF con la query de SQL
    CreadorDF_vent_marcas(Marca)
    #Creo y entreno el Modelo y se lo asigno a la variable LR
    LR = LinearRegression().fit(df['Año'].values.reshape(-1,1),df['Cantidad Venta'])
    #Informo al usuario que ahora debe elegir el año que desea conocer y le doy algunas advertencias
    #Informo al usuario que ahora debe elegir el año que desea conocer y le doy algunas advertencias
    print('¿Ahora que año queres pronosticar?')
    ('------------------------------------------------------------------------------------------------------------')
    print('Toma en cuenta que mientras mas lejos del ultimo año que se tienen datos, mas especulativos seran los datos')
    print('Ademas que mientras menos datos se tenga es mas probable que la pronosticacion sea inexacta')
    ('------------------------------------------------------------------------------------------------------------')
    print('Aca te muestro la lista de los años que tenemos datos de esta Marca y sus valores')
    #Muestro el DF que traje para que el usuario conozca con lo que esta trabajando
    print(str(df))
    #Hago que el usuario introduzca el Año que desea predecir
    A = int(input('Introduce el año a predecir: '))
    #Creo la funcion que predice el año con una regresion lineal en base a los datos obtenidos del entrenamineto
    y_pred = LR.intercept_ + LR.coef_* A
    #Muestro el resultado de esa operacion
    print('La estimacion para el Año {} es de: '.format(A)+str(int(y_pred)) + ' Ventas en total para la Marca {}'.format(Marca))
elif Elec == 4:
    print('Eligío 4 Cantidad de ventas por articulo')
    print('-----------------------------------------------------------------------')
    Articulo = str(input('Ingrese el numero del Articulo a pronosticar: '))
    CreadorDF_vent_articulo(Articulo)
    #Creo y entreno el Modelo y se lo asigno a la variable LR
    LR = LinearRegression().fit(df['Año'].values.reshape(-1,1),df['Ventas'])
    #Informo al usuario que ahora debe elegir el año que desea conocer y le doy algunas advertencias
    #Informo al usuario que ahora debe elegir el año que desea conocer y le doy algunas advertencias
    print('¿Ahora que año queres pronosticar?')
    ('------------------------------------------------------------------------------------------------------------')
    print('Toma en cuenta que mientras mas lejos del ultimo año que se tienen datos, mas especulativos seran los datos')
    print('Ademas que mientras menos datos se tenga es mas probable que la pronosticacion sea inexacta')
    ('------------------------------------------------------------------------------------------------------------')
    print('Aca te muestro la lista de los años que tenemos datos de este Articulo y sus valores')
    #Muestro el DF que traje para que el usuario conozca con lo que esta trabajando
    print(str(df))
    #Hago que el usuario introduzca el Año que desea predecir
    A = int(input('Introduce el año a predecir: '))
    #Creo la funcion que predice el año con una regresion lineal en base a los datos obtenidos del entrenamineto
    y_pred = LR.intercept_ + LR.coef_* A
    #Muestro el resultado de esa operacion
    print('La estimacion para el Año {} es de: '.format(A)+str(int(y_pred)) + ' Ventas en total para el Articulo {}'.format(Articulo))
else:
    print('Introducjo un valor que no reconozco')

-----------------------------------------------------------------------
                            Bienvenido
-----------------------------------------------------------------------
-Este programa Puede Realizar 4 pronosticaciones.
-----------------------------------------------------------------------
-1 Cantidad de ventas por Vendedor.
-----------------------------------------------------------------------
-2 Cantidad de ventas por Sucursal.
-----------------------------------------------------------------------
-3 Cantidad de ventas por Marca.
-----------------------------------------------------------------------
-4 Cantidad de ventas por Articulo.
-----------------------------------------------------------------------
Respectivamente el programa las reconoce a las mismas como 1, 2, 3, 4.
-----------------------------------------------------------------------
Eligío 3 Cantidad de ventas por Marca
-----------------------------------------------------------------------
¿Ahora que añ